# Instructions

To run this notebook you will need to:
1. Download Ollam from [here](https://ollama.com/)
2. Go to your terminal and type: `ollama pull llama3` to download llama3 model in your local computer.

## Imports

In [1]:
import subprocess
import re
import ast
import pandas as pd

## Function to extract location name

In [2]:
def extract_locations_from_text(text, model="llama3"):
    """
    Directly extracts the city name from the given court text.

    Parameters
    ----------
    text : str
        The court text to analyze.
    model : str, optional
        The name of the language model to use (default is "llama3").

    Returns
    -------
    str
        The extracted city name, or 'NA' if no location is found.
    """
    prompt = f"""
You are a legal assistant. Identify the city where the case was heard from the following court text.

Instructions:
- Extract the city where the case was heard.
- Just provide the city name, nothing else.
- If no location is found, return NA

Court Text:
{text}
"""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    output = result.stdout.decode().strip()
    
    return output if output else "NA"

def extract_locations_from_dataframe(df, startline, endline, text_column="unofficial_text", model="llama3"):
    """
    Extract city names from a DataFrame containing court texts.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court text data.
    startline : int
        The starting line number of the text to consider.
    endline : int
        The ending line number of the text to consider.
    text_column : str, optional
        The column in `df` that contains the court text (default is "unofficial_text").
    model : str, optional
        The language model to use (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A copy of the input DataFrame with an additional 'locations' column containing extracted city names.
    """
    df = df.copy()

    def extract_slice_and_location(text):
        lines = text.splitlines()
        selected_text = "\n".join(lines[startline:endline])
        return extract_locations_from_text(selected_text, model=model)

    df["locations"] = df[text_column].apply(extract_slice_and_location)

    return df



## FCA Data

In [3]:
FCA = pd.read_csv("../data/processed/FCA_judges.csv")
FCA

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
0,0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']"
1,1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']"
2,2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","['Dawson', 'Trudel', 'Near']"
3,3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","['Nadon', 'Stratas', 'Scott']"
4,4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"['Gauthier', 'Mainville', 'Boivin']"
...,...,...,...,...,...,...,...,...,...,...
94,94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"['PELLETIER', 'WOODS', 'ROUSSEL']"
95,95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"['Webb', 'Mactavish', 'Walker']"
96,96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","['Stratas', 'Boivin', 'Heckman']"
97,97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"['Boivin', 'De Montigny', 'Leblanc']"


In [4]:
FCA_city = extract_locations_from_dataframe(FCA, 20, 55,  text_column="unofficial_text", model="llama3")
FCA_city

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges,locations
0,0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto
1,1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto
2,2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","['Dawson', 'Trudel', 'Near']",Vancouver
3,3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","['Nadon', 'Stratas', 'Scott']",Vancouver
4,4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"['Gauthier', 'Mainville', 'Boivin']",Montréal
...,...,...,...,...,...,...,...,...,...,...,...
94,94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"['PELLETIER', 'WOODS', 'ROUSSEL']",Ottawa
95,95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"['Webb', 'Mactavish', 'Walker']",Ottawa
96,96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","['Stratas', 'Boivin', 'Heckman']",Ottawa
97,97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"['Boivin', 'De Montigny', 'Leblanc']",Ottawa


In [5]:
FCA_city['locations'].value_counts()

locations
Ottawa        36
Toronto       33
Vancouver     11
Montréal       8
Winnipeg       4
Montreal       3
Calgary        2
Saint Paul     1
NA             1
Name: count, dtype: int64

In [6]:
FCA_city.to_csv("../data/processed/FCA_city.csv")

## FC Data

In [9]:
FC = pd.read_csv("../data/processed/FC_Judges.csv")
FC

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
0,11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi...",['Strickland']
1,11930,2014 FC 1008,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Bautista v. Canada (Citizenship and Immigratio...,['misrepresentation'],['Diner']
2,11933,2014 FC 1011,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canada (Public Safety and Emergency Preparedne...,['non_compliance'],"['Shore', 'Gauthier']"
3,11934,2014 FC 1012,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Po v. Canada (Citizenship and Immigration)\nCo...,['human_rights'],['Tremblay-Lamer']
4,11935,2014 FC 1015,FC,2014,en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Gordon v. Canada (Citizenship and Immigration)...,"['human_rights', 'misrepresentation']",['Locke']
...,...,...,...,...,...,...,...,...,...,...
4826,23170,2022 CF 882,FC,2022,fr,2022-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Jules c. Canada (Citoyenneté et Immigration)\n...,"['human_rights', 'serious_criminality', 'crimi...",['St-Louis']
4827,23173,2023 CF 1619,FC,2023,fr,2023-12-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salhi c. Canada (Citoyenneté et Immigration)\n...,"['serious_criminality', 'criminality']",['Azmudeh']
4828,23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']",['Heneghan']
4829,23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights'],"['Ngo', 'Tabib', 'Steele']"


In [ ]:
FC_city = extract_locations_from_dataframe(FCA, 10, 25,  text_column="unofficial_text", model="llama3")
FC_city

In [ ]:
FC_city['locations'].value_counts()

In [ ]:
FC_city.to_csv("../data/processed/FC_city.csv")